In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.c

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-27 02:27:56--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.96MB/s    in 0.2s    

2022-06-27 02:27:56 (4.96 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
electronics_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
electronics_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [6]:
# Vine table to DF
vine_df = electronics_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|          1|            0|          0|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R20D9EHB7N20V6|          5|            0|          0|   N|                Y|
|R1WUTD8MVSROJU|          5|            0|          0|   N|                Y|
|R1QCYLT25812DM|          4|            0|          0|   N|                Y|
| R904DQPBCEM7A|          4|            0|          0|   N|     

In [7]:
# Retreive all rows where total_votes count is equal or greater than 20
votes20_vine_df = vine_df.filter("total_votes >= 20")
votes20_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [9]:
#step 2
helpful_df = vine_df.filter("helpful_votes/total_votes >= .50")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R1WI5NISM6GAUG|          2|            4|          5|   N|                Y|
|R27F4OF4BIA4LU|          2|            1|          1|   N|                Y|
|R1DMFKFFMO6DYB|          4|            1|          1|   N|                Y|
| RKBT2Y4X8X9XT|          5|            2|          2|   N|                Y|
|R20TC495KA8WVA|          1|            1|          1|   N|                Y|
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R2MXU8IVR1Q1S5|          5|            1|          1|   N|     

In [13]:
#step 3
vine_paid_df = helpful_df.filter(helpful_df['vine']=='Y')
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1T7X1RPWYTRBU|          5|            2|          4|   Y|                N|
|R2N4GVP3Q5PF9Y|          3|            2|          3|   Y|                N|
| RCAFL1X52MFZM|          5|            1|          2|   Y|                N|
|R17D8E4KO0FS83|          5|            1|          1|   Y|                N|
|R184FOUNZZ7KO8|          5|           15|         20|   Y|                N|
|R2AYZV2RYCKIDO|          4|            1|          1|   Y|                N|
|R2PLGJM0MUI41A|          5|            1|          2|   Y|                N|
|R2B2RYH8VXXFNO|          5|            9|         10|   Y|                N|
|R2WMMK2LY6SJV3|          3|            2|          3|   Y|                N|
|R3JLX3WLHL46OD|          5|           11|         13|   Y|     

In [14]:
#step 4
vine_unpaid_df = helpful_df.filter(helpful_df['vine']=='N')
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R1WI5NISM6GAUG|          2|            4|          5|   N|                Y|
|R27F4OF4BIA4LU|          2|            1|          1|   N|                Y|
|R1DMFKFFMO6DYB|          4|            1|          1|   N|                Y|
| RKBT2Y4X8X9XT|          5|            2|          2|   N|                Y|
|R20TC495KA8WVA|          1|            1|          1|   N|                Y|
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R2MXU8IVR1Q1S5|          5|            1|          1|   N|     

In [19]:
# Total number of  reviews
total_helpful_reviews = helpful_df.count()
print("Number of helpful reviews %f" % total_helpful_reviews)


Number of helpful reviews 936994.000000


In [17]:
# Total number of paid reviews
total_number_of_paid_reviews = vine_paid_df.count()

# Total number of unpaid reviews
total_number_of_unpaid_reviews = vine_unpaid_df.count()

print("Number of paid helpful reviews %f" % total_number_of_paid_reviews)
print("Number of unpaid helpful reviews %f" % total_number_of_unpaid_reviews)

Number of paid helpful reviews 10259.000000
Number of unpaid helpful reviews 926735.000000


In [18]:
# Total number of 5 star paid reviews
star5_paid_reviews = vine_paid_df.filter(vine_paid_df['star_rating']==5).count()

# Total number of 5 star unpaid reviews
star5_unpaid_reviews = vine_unpaid_df.filter(vine_unpaid_df['star_rating']==5).count()

print("Number of 5 star paid helpful reviews %f" % star5_paid_reviews)
print("Number of 5 star unpaid helpful reviews %f" % star5_unpaid_reviews)

Number of 5 star paid helpful reviews 4603.000000
Number of 5 star unpaid helpful reviews 482639.000000


In [27]:
# Percentage of of 5 star paid reviews
percentage_5star_paid = (star5_paid_reviews/total_number_of_paid_reviews)*100
percentage_5star_paid

44.867920849985374

In [29]:
# Percentage of of 5 star unpaid reviews
percentage_5star_unpaid = (star5_unpaid_reviews/total_number_of_unpaid_reviews)*100
percentage_5star_unpaid

52.07950492859339

In [30]:
print("Number of helpful reviews %f" % total_helpful_reviews)
print("Number of paid helpful reviews %f" % total_number_of_paid_reviews)
print("Number of unpaid helpful reviews %f" % total_number_of_unpaid_reviews)
print("Number of 5 star paid helpful reviews %f" % star5_paid_reviews)
print("Number of 5 star unpaid helpful reviews %f" % star5_unpaid_reviews)

Number of helpful reviews 936994.000000
Number of paid helpful reviews 10259.000000
Number of unpaid helpful reviews 926735.000000
Number of 5 star paid helpful reviews 4603.000000
Number of 5 star unpaid helpful reviews 482639.000000
